In [69]:
import pandas as pd
import requests

data = pd.read_csv('311_cases_clean.csv')
data['Opened'] = pd.to_datetime(data['Opened'])
data = data[data['Opened'] > '12-31-17']
all_results = []

for index, row in data.iterrows():
    
    # get results from FCC API and store as a dataframe
    r = requests.get('https://geo.fcc.gov/api/census/area?lat={}&lon={}&format=json'.format(row['Latitude'],
                                                                                            row['Longitude']))
    
    try:
        results = r.json()['results']
        for result in results:
            result['CaseID'] = row['CaseID']
        all_results.append(results)
        
    except:
        pass

In [70]:
pd.DataFrame(sum(all_results, [])).to_csv('caseid_block_2018.csv', index = False)